In [1]:
import itertools
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.distributions as D
from torch.utils.data import DataLoader, TensorDataset

n = 5
epochs = 3
supervisionEpochs = 3
lr = 0.001
log_interval = 20
trainSize = 100000
penaltyLambda = 10
doublePeakHighMean = 0.9
doublePeakLowMean = 0.1
doublePeakStd = 0.1
uniformlow=0
uniformhigh=0.5
normalloc = 0.2
normalscale = 0.1
order="uniform";
order1=2;
state=["twopeak","normal","uniform"]


d1 = D.normal.Normal(doublePeakLowMean, doublePeakStd)
d2 = D.normal.Normal(doublePeakHighMean, doublePeakStd)
distributionRatio = (d1.cdf(1) + d2.cdf(1) - d1.cdf(0) - d2.cdf(0)) / 2
distributionBase = d1.cdf(0) + d2.cdf(0)

d3 = D.normal.Normal(normalloc, normalscale)
d4 = D.uniform.Uniform(uniformlow,uniformhigh)


def cdf(x,y, i=None):
    if(y=="twopeak"):
        return (d1.cdf(x) + d2.cdf(x) - distributionBase) / 2 / distributionRatio
    elif(y=="normal"):
        return d3.cdf(x);
    elif(y=="uniform"):
        return d4.cdf(x);


# def cdf(x, i=None):
#     if x < 0.1:
#         return 0
#     if x <= 0.2:
#         return 0.5 * (x - 0.1) / 0.1
#     if x < 0.8:
#         return 0.5
#     if x < 0.9:
#         return 0.5 + 0.5 * (x - 0.8) / 0.1
#     return 1


print(distributionBase)
print(d3)
print(d4)

tensor(0.1587)
Normal(loc: 0.20000000298023224, scale: 0.10000000149011612)
Uniform(low: 0.0, high: 0.5)


In [2]:

print(order)

uniform


In [3]:
if(order==state[0]):
    signals = np.random.randint(2, size=(trainSize, n))
# samples1 = np.random.uniform(low=0.1, high=0.2, size=(trainSize, n))
# samples2 = np.random.uniform(low=0.8, high=0.9, size=(trainSize, n))
    samples1 = np.random.normal(
        loc=doublePeakLowMean, scale=doublePeakStd, size=(trainSize, n)
    )
    for i in range(trainSize):
        for j in range(n):
            while samples1[i, j] < 0 or samples1[i, j] > 1:
                samples1[i, j] = np.random.normal(
                    loc=doublePeakLowMean, scale=doublePeakStd
                )
    samples2 = np.random.normal(
        loc=doublePeakHighMean, scale=doublePeakStd, size=(trainSize, n)
    )
    for i in range(trainSize):
        for j in range(n):
            while samples2[i, j] < 0 or samples2[i, j] > 1:
                samples2[i, j] = np.random.normal(
                    loc=doublePeakHighMean, scale=doublePeakStd
                )
    samplesJoint = signals * samples1 - (signals - 1) * samples2
    tp_tensor = torch.tensor(samplesJoint, dtype=torch.float32)
    # tp_tensor = torch.tensor(np.random.rand(10000, n), dtype=torch.float32)
elif(order==state[1]):
    samples1 = np.random.normal(
        loc=normalloc, scale=normalscale, size=(trainSize, n)
    )
    for i in range(trainSize):
        for j in range(n):
            while samples1[i, j] < 0 or samples1[i, j] > 1:
                samples1[i, j] = np.random.normal(
                    loc=normalloc, scale=normalscale
                )
    samplesJoint = samples1
    tp_tensor = torch.tensor(samplesJoint, dtype=torch.float32)
elif(order==state[2]):   
    samples1 = np.random.uniform(
        uniformlow, uniformhigh, size=(trainSize, n)
    )
    for i in range(trainSize):
        for j in range(n):
            while samples1[i, j] < 0 or samples1[i, j] > 1:
                samples1[i, j] = np.random.normal(
                    uniformlow, uniformhigh
                )
    samplesJoint = samples1
    tp_tensor = torch.tensor(samplesJoint, dtype=torch.float32)
tp_dataset = TensorDataset(tp_tensor[: round(trainSize * 0.3)])
tp_dataset_testing = TensorDataset(tp_tensor[round(trainSize * 0.7) :])
tp_dataloader = DataLoader(tp_dataset, batch_size=128, shuffle=True)
tp_dataloader_testing = DataLoader(tp_dataset_testing, batch_size=256, shuffle=False)


# for mapping binary to payments before softmax
model = nn.Sequential(
    nn.Linear(n, 100),
    nn.ReLU(),
    nn.Linear(100, 100),
    nn.ReLU(),
    nn.Linear(100, 100),
    nn.ReLU(),
    nn.Linear(100, n),
)


# optimizer = optim.SGD(model.parameters(), lr=lr)
optimizer = optim.Adam(model.parameters(), lr=lr)


In [4]:
def bitsToPayments(bits):
    if sum(bits).item() == 0:
        return torch.ones(n)
    bits = bits.type(torch.float32)
    negBits = torch.ones(n) - bits
    payments = model(bits)
    payments = payments - 1000 * negBits
    payments = torch.softmax(payments, 0)
    payments = payments + negBits
    return payments


def tpToBits(tp, bits=torch.ones(n).type(torch.uint8)):
    payments = bitsToPayments(bits)
    newBits = (tp >= payments).type(torch.uint8)
    
    if torch.equal(newBits, bits):
        return bits
    else:
        return tpToBits(tp, newBits)#bits-bits#tpToBits(tp, newBits)


def tpToPayments(tp):
    return bitsToPayments(tpToBits(tp))


def tpToTotalDelay(tp):
    return n - sum(tpToBits(tp).type(torch.float32))


In [5]:
print(1_000_000)
print(cdf(0.5,order))

1000000
tensor(1.)


In [6]:
dpPrecision = 100
# howManyPpl left, money left, yes already
dp = np.zeros([n + 1, dpPrecision + 1, n + 1])
decision = np.zeros([n + 1, dpPrecision + 1, n + 1], dtype=np.uint8)
# ppl = 0 left
for yes in range(n + 1):
    for money in range(dpPrecision + 1):
        if money == 0:
            dp[0, 0, yes] = 0
        else:
            dp[0, money, yes] = yes# + 1.0
for ppl in range(1, n + 1):
    for yes in range(n + 1):
        for money in range(dpPrecision + 1):
            minSoFar = 1_000_000
            for offerIndex in range(money + 1):
                offer = offerIndex / dpPrecision
                res = (1 - cdf(offer,order)) * dp[
                    ppl - 1, money - offerIndex, min(yes + 1, n)
                ] + cdf(offer,order) * (1 + dp[ppl - 1, money, yes])
                if minSoFar > res:
                    minSoFar = res
                    decision[ppl, money, yes] = offerIndex
            dp[ppl, money, yes] = minSoFar


In [7]:
print(dp[n, dpPrecision, 0])

4.052292823791504


In [8]:
# howManyPpl left, money left, yes already
dp_H = np.zeros([n + 1 , n + 1, dpPrecision + 1])
decision_H = np.zeros([n + 1 , n + 1, dpPrecision + 1], dtype=np.uint8)
# ppl = 0 left
for i in range(1,n+1):
    for money in range(dpPrecision + 1):
        if money == 0:
            dp_H[i, 0, 0] = 1
        else:
            offer = money / dpPrecision
            dp_H[i, 0, money] = 0#cdf(offer)# + 1.0
for i in range(1,n+1):
    for ppl in range(1, i + 1):
        for money in range(dpPrecision + 1):
            maxSoFar = -1_000_000
            for offerIndex in range(money + 1):
                offer = offerIndex / dpPrecision
                res = (1-cdf(offer,order)) * dp_H[
                     i, ppl - 1, money - offerIndex
                    ]
                if maxSoFar < res:
                    maxSoFar = res
                    decision_H[i, ppl, money] = offerIndex
            dp_H[i, ppl, money] = maxSoFar
    print("\n",i)
    print(dp_H[i, i, dpPrecision])
    print(5*(1-dp_H[i, i, dpPrecision]))


 1
0.0
5.0

 2
0.0
5.0

 3
0.036991994827985764
4.815040025860071

 4
0.0625
4.6875

 5
0.0777600109577179
4.6111999452114105


In [9]:
print(samplesJoint)

[[0.00769991 0.213836   0.35532296 0.32894044 0.42415031]
 [0.46662239 0.2530678  0.39280985 0.1024852  0.24375605]
 [0.08780481 0.40089717 0.26479932 0.16782505 0.2533314 ]
 ...
 [0.44957855 0.18671957 0.03793543 0.17496262 0.39187532]
 [0.49565617 0.11346518 0.10765269 0.4014891  0.35374696]
 [0.10963361 0.26955397 0.3158091  0.30524238 0.07572215]]


In [10]:
def plan_dp(temp):
    #print(temp)
    remain=dpPrecision
    yes=0;
    ans =0;
    o_list=[];
    remain_list=[];
    for ppl in range(n,0,-1):
        o=decision[ppl, remain, yes]
        #print(o,remain)
        o_list.append(o)
        remain_list.append(remain);
        if(o<temp[n-ppl]):
            remain-=int(o);
            yes+=1;
        elif (remain>0):
            ans+=1;
    if(remain<=0):
        return ans,o_list;
    else:
        return n,o_list;
    

In [11]:
ans_list=[];
for i in range(10000):
    temp=samplesJoint[i]*dpPrecision
    #print(temp)
    ans_list.append(plan_dp(temp)[0]);
    #print("\n",temp)
    #print(plan_dp(temp))
print(sum(ans_list)/len(ans_list))

4.0674


In [12]:
def dpSupervisionRule(tp):
    tp = list(tp.numpy())
    bits = [1 for ii in range(n)]
    payments = [0 for ii in range(n)]
    money = dpPrecision
    yes = 0
    for i in range(n):
        offerIndex = decision[n - i, money, yes]
        offer = offerIndex / dpPrecision
        if tp[i] >= offer:
            money -= offerIndex
            yes += 1
            bits[i] = 1
            payments[i] = offer
        else:
            bits[i] = 0
            payments[i] = 1
    if money > 0:
        bits = [0 for ii in range(n)]
        payments = [1 for ii in range(n)]
    bits = torch.tensor(bits, dtype=torch.uint8)
    payments = torch.tensor(payments, dtype=torch.float32)
    # print()
    # print(tp)
    # print(bits)
    # print(payments)
    # print()
    return (bits, payments)


def heuristicSupervisionRule(tp):
    tp = list(tp.numpy())
    bits = [1 for ii in range(n)]
    payments = [0 for ii in range(n)]
    tempold=-1;
    for turns in range(n,0,-1):
        money = dpPrecision
        j=0
        tempo=sum(bits)
        #print("bits",tempo)
        for i in range(n):
            if(j>=n):
                break;
            
            offerIndex = decision_H[tempo ,tempo - i, money]
            offer = offerIndex / dpPrecision
            while(j<n):
                if(bits[j]!=0):
                    break;
                j+=1;
            if(j>=n):
                break;
            if tp[j] >= offer:
                #print(money,j,tp[j],offer)
                money -= offerIndex
                bits[j] = 1
                payments[j] = offer
            else:
                bits[j] = 0;
                payments[j] = 1;
            j+=1;
        #print("money",money)
        if(money==0 and tempold==tempo):
            break;
        tempold=tempo;
    if money > 0:
        bits = [0 for ii in range(n)]
        payments = [1 for ii in range(n)]

    bits = torch.tensor(bits, dtype=torch.float32)
    payments = torch.tensor(payments, dtype=torch.float32)
    # print()
    # print(tp)
    # print(bits)
    # print(payments)
    # print()
    return (bits, payments)



def costSharingSupervisionRule(tp):
    tp = list(tp.numpy())
    for k in range(n, -1, -1):
        if k == 0:
            break
        bits = [1 if tp[ii] >= 1 / k else 0 for ii in range(n)]
        if sum(bits) == k:
            break
    if k == 0:
        payments = [1 for ii in range(n)]
    else:
        payments = [1 / k if bits[ii] == 1 else 1 for ii in range(n)]
    bits = torch.tensor(bits, dtype=torch.uint8)
    payments = torch.tensor(payments, dtype=torch.float32)
    return (bits, payments)


def costSharingDelay(tp):
    return n - sum(costSharingSupervisionRule(tp)[0]).item()

def dpDelay(tp):
    return n - sum(dpSupervisionRule(tp)[0]).item()

def heuristicDelay(tp):
    return n - sum(heuristicSupervisionRule(tp)[0]).item()


templire=0;
num=1
for i in range(num):
    temp=torch.tensor(samplesJoint[i])
    #print(temp)
    #print(temp);
    #print(heuristicSupervisionRule(temp))
    #print(dpSupervisionRule(temp))
    #print(dpDelay(temp))
    res=dpDelay(temp)
    templire+=res
    #print("delay",res)
    #print()
print(templire/num)

tensor([0.0077, 0.2138, 0.3553, 0.3289, 0.4242], dtype=torch.float64)
(tensor([0., 0., 0., 0., 0.]), tensor([1., 1., 1., 1., 1.]))

tensor([0.4666, 0.2531, 0.3928, 0.1025, 0.2438], dtype=torch.float64)
(tensor([0., 0., 0., 0., 0.]), tensor([1., 1., 1., 1., 1.]))

tensor([0.0878, 0.4009, 0.2648, 0.1678, 0.2533], dtype=torch.float64)
(tensor([0., 0., 0., 0., 0.]), tensor([1., 1., 1., 1., 1.]))

tensor([0.0566, 0.2243, 0.0862, 0.1015, 0.1942], dtype=torch.float64)
(tensor([0., 0., 0., 0., 0.]), tensor([1., 1., 1., 1., 1.]))

tensor([0.0928, 0.2699, 0.3987, 0.4980, 0.4970], dtype=torch.float64)
(tensor([0., 1., 1., 1., 1.]), tensor([1.0000, 0.2500, 0.2500, 0.2500, 0.2500]))



In [13]:
for i in range(n, 0, -1):
            print([1 if ii >= i else 0 for ii in range(n)])
            print(
                    tpToPayments(
                            torch.tensor([0 if ii >= i else 1 for ii in range(n)], dtype=torch.float32)
                    )
                )

[0, 0, 0, 0, 0]
tensor([0.2072, 0.2059, 0.2030, 0.2228, 0.1612], grad_fn=<AddBackward0>)
[0, 0, 0, 0, 1]
tensor([0.2481, 0.2446, 0.2446, 0.2626, 1.0000], grad_fn=<AddBackward0>)
[0, 0, 0, 1, 1]
tensor([0.3451, 0.3270, 0.3279, 1.0000, 1.0000], grad_fn=<AddBackward0>)
[0, 0, 1, 1, 1]
tensor([0.5081, 0.4919, 1.0000, 1.0000, 1.0000], grad_fn=<AddBackward0>)
[0, 1, 1, 1, 1]
tensor([1., 1., 1., 1., 1.], grad_fn=<AddBackward0>)


In [14]:
allBits = [torch.tensor(bits) for bits in itertools.product([0, 1], repeat=n)]
print(allBits)

for batch_idx, (tp_batch,) in enumerate(tp_dataloader_testing):
    penalty = 0
    for bitsMoreOnes in allBits:
        for i in range(n):
            if bitsMoreOnes[i] == 1:
                bitsLessOnes = bitsMoreOnes.clone()
                bitsLessOnes[i] = 0
                penalty = penalty + sum(
                        torch.relu(
                            bitsToPayments(bitsMoreOnes) - bitsToPayments(bitsLessOnes)
                        )
                    )
    loss = penalty * penaltyLambda
    for tp in tp_batch:
            for i in range(n):
                tp1 = tp.clone()
                tp1[i] = 1
                tp0 = tp.clone()
                tp0[i] = 0
                offer = tpToPayments(tp1)[i]
                delay1 = tpToTotalDelay(tp1)
                delay0 = tpToTotalDelay(tp0)
                #loss = loss + (1 - cdf(offer)) * delay1 + cdf(offer) * delay0
                loss = loss + (1 - cdf(offer,order)) * delay1 + cdf(offer,order) * delay0
    print()
    print(tp)
    tp1 = tp.clone()
    tp1[0] = 1
    tp0 = tp.clone()
    tp0[0] = 0
    offer = tpToPayments(tp1)[0]
    print(offer)
    print(tpToPayments(tp1))
    print(delay1)
    print(delay0)
    break;
#print(loss)
#print(penalty)

[tensor([0, 0, 0, 0, 0]), tensor([0, 0, 0, 0, 1]), tensor([0, 0, 0, 1, 0]), tensor([0, 0, 0, 1, 1]), tensor([0, 0, 1, 0, 0]), tensor([0, 0, 1, 0, 1]), tensor([0, 0, 1, 1, 0]), tensor([0, 0, 1, 1, 1]), tensor([0, 1, 0, 0, 0]), tensor([0, 1, 0, 0, 1]), tensor([0, 1, 0, 1, 0]), tensor([0, 1, 0, 1, 1]), tensor([0, 1, 1, 0, 0]), tensor([0, 1, 1, 0, 1]), tensor([0, 1, 1, 1, 0]), tensor([0, 1, 1, 1, 1]), tensor([1, 0, 0, 0, 0]), tensor([1, 0, 0, 0, 1]), tensor([1, 0, 0, 1, 0]), tensor([1, 0, 0, 1, 1]), tensor([1, 0, 1, 0, 0]), tensor([1, 0, 1, 0, 1]), tensor([1, 0, 1, 1, 0]), tensor([1, 0, 1, 1, 1]), tensor([1, 1, 0, 0, 0]), tensor([1, 1, 0, 0, 1]), tensor([1, 1, 0, 1, 0]), tensor([1, 1, 0, 1, 1]), tensor([1, 1, 1, 0, 0]), tensor([1, 1, 1, 0, 1]), tensor([1, 1, 1, 1, 0]), tensor([1, 1, 1, 1, 1])]

tensor([0.1265, 0.4438, 0.2564, 0.2627, 0.2017])
tensor(0.2072, grad_fn=<SelectBackward>)
tensor([0.2072, 0.2059, 0.2030, 0.2228, 0.1612], grad_fn=<AddBackward0>)
tensor(4.)
tensor(5.)


In [15]:
allBits = [torch.tensor(bits) for bits in itertools.product([0, 1], repeat=n)]

runningLossNN = []
runningLossCS = []
runningLossDP = []
runningLossHeuristic = []

def recordAndReport(name, source, loss, n=100):
    source.append(loss)
    realLength = min(n, len(source))
    avgLoss = sum(source[-n:]) / realLength
    print(f"{name} ({realLength}): {avgLoss}")


def supervisionTrain(epoch, supervisionRule):
    model.train()
    for batch_idx, (tp_batch,) in enumerate(tp_dataloader):
        optimizer.zero_grad()
        loss = 0
        for tp in tp_batch:
            bits, payments = supervisionRule(tp)
            # print()
            # print("supervision")
            # print(tp)
            # print(bits)
            # print()
            # print(payments)
            # print(bitsToPayments(bits))
            # print()
            loss = loss + F.mse_loss(bitsToPayments(bits), payments)

        loss = loss / len(tp_batch)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            #print()
            #print(tp)
            #print(bits)
            #print(payments)
            #print(bitsToPayments(bits))
            print(
                "Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                    epoch,
                    batch_idx * len(tp_batch),
                    len(tp_dataloader.dataset),
                    100.0 * batch_idx / len(tp_dataloader),
                    loss.item(),
                )
            )


def train(epoch):
    model.train()
    for batch_idx, (tp_batch,) in enumerate(tp_dataloader):
        optimizer.zero_grad()
        penalty = 0
        for bitsMoreOnes in allBits:
            for i in range(n):
                if bitsMoreOnes[i] == 1:
                    bitsLessOnes = bitsMoreOnes.clone()
                    bitsLessOnes[i] = 0
                    penalty = penalty + sum(
                        torch.relu(
                            bitsToPayments(bitsMoreOnes) - bitsToPayments(bitsLessOnes)
                        )
                    )
        loss = penalty * penaltyLambda
        # costSharingLoss = 0
        # dpLoss = 0
        for tp in tp_batch:
            # costSharingLoss += costSharingDelay(tp)
            # dpLoss += dpDelay(tp)
            # print()
            # print("---")
            # print(tp)
            # print(costSharingSupervisionRule(tp))
            # print(dpSupervisionRule(tp))
            # print(costSharingDelay(tp), dpDelay(tp))
            # print()
            for i in range(n):
                tp1 = tp.clone()
                tp1[i] = 1
                tp0 = tp.clone()
                tp0[i] = 0
                offer = tpToPayments(tp1)[i]
                delay1 = tpToTotalDelay(tp1)
                delay0 = tpToTotalDelay(tp0)
                loss = loss + (1 - cdf(offer,order)) * delay1 + cdf(offer,order) * delay0

        loss = loss / len(tp_batch) / n
        # costSharingLoss /= len(tp_batch)
        # dpLoss /= len(tp_batch)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print(
                "Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                    epoch,
                    batch_idx * len(tp_batch),
                    len(tp_dataloader.dataset),
                    100.0 * batch_idx / len(tp_dataloader),
                    loss.item(),
                )
            )
            # recordAndReport("NN", runningLossNN, loss.item())
            # recordAndReport("CS", runningLossCS, costSharingLoss)
            # recordAndReport("DP", runningLossDP, dpLoss)
            # print(dp[n, dpPrecision, 0])
            print("penalty:",penalty.item())
            # print(distributionRatio)
            # for i in range(n, 0, -1):
            #     print(
            #         tpToPayments(
            #             torch.tensor([1] * i + [0] * (n - i), dtype=torch.float32)
            #         )
            #     )


In [16]:
def test():
    model.eval()
    with torch.no_grad():
        costSharingLoss = 0
        dpLoss = 0
        nnLoss = 0
        heuristicLoss=0
        lenLoss= 0
        for (tp_batch,) in tp_dataloader_testing:

            for tp in tp_batch:
                costSharingLoss += costSharingDelay(tp)
                dpLoss += dpDelay(tp)
                nnLoss += tpToTotalDelay(tp)
                heuristicLoss+= heuristicDelay(tp)
            lenLoss+=len(tp_batch)
        costSharingLoss /= lenLoss
        dpLoss /= lenLoss
        nnLoss /= lenLoss
        heuristicLoss /= lenLoss
        #print(lenLoss)
        recordAndReport("NN", runningLossNN, nnLoss)
        recordAndReport("CS", runningLossCS, costSharingLoss)
        recordAndReport("DP", runningLossDP, dpLoss)
        recordAndReport("heuristic", runningLossHeuristic, heuristicLoss)
        print("DP:",dp[n, dpPrecision,0])
        #for i in range(n, 0, -1):
        #    print("Heuristic:",i,5*(1-dp_H[i, i, dpPrecision]))
        for i in range(n, 0, -1):
            print(
                    tpToPayments(
                            torch.tensor([0 if ii >= i else 1 for ii in range(n)], dtype=torch.float32)
                    )
                )

#model=torch.load("save/pytorchNN=5dp1");
#model.eval()
for epoch in range(1, supervisionEpochs + 1):
    print("distributionRatio",distributionRatio)
    if(order1==0):
        supervisionTrain(epoch, costSharingSupervisionRule)
    elif(order1==1):
        supervisionTrain(epoch, dpSupervisionRule)
    elif(order1==2):
        supervisionTrain(epoch, heuristicSupervisionRule)
    
    
temp=torch.tensor(samplesJoint[0])
print(temp);
if(order1==0):
    bits, payments = costSharingSupervisionRule(temp)
    print(costSharingSupervisionRule(temp))
    print(bitsToPayments(bits))
if(order1==1):
    bits, payments = dpSupervisionRule(temp)
    print(dpSupervisionRule(temp))
    print(bitsToPayments(bits))
if(order1==2):
    bits, payments = heuristicSupervisionRule(temp)
    print(heuristicSupervisionRule(temp))
    print(bitsToPayments(bits))
print()
test()

for epoch in range(1, epochs + 1):
    train(epoch)
    test()


distributionRatio tensor(0.8413)
Train Epoch: 1 [0/30000 (0%)]	Loss: 0.000080
Train Epoch: 1 [2560/30000 (9%)]	Loss: 0.000001
Train Epoch: 1 [5120/30000 (17%)]	Loss: 0.000000
Train Epoch: 1 [7680/30000 (26%)]	Loss: 0.000000
Train Epoch: 1 [10240/30000 (34%)]	Loss: 0.000000
Train Epoch: 1 [12800/30000 (43%)]	Loss: 0.000000
Train Epoch: 1 [15360/30000 (51%)]	Loss: 0.000000
Train Epoch: 1 [17920/30000 (60%)]	Loss: 0.000000
Train Epoch: 1 [20480/30000 (68%)]	Loss: 0.000000
Train Epoch: 1 [23040/30000 (77%)]	Loss: 0.000000
Train Epoch: 1 [25600/30000 (85%)]	Loss: 0.000000
Train Epoch: 1 [28160/30000 (94%)]	Loss: 0.000000
distributionRatio tensor(0.8413)
Train Epoch: 2 [0/30000 (0%)]	Loss: 0.000000
Train Epoch: 2 [2560/30000 (9%)]	Loss: 0.000000
Train Epoch: 2 [5120/30000 (17%)]	Loss: 0.000000
Train Epoch: 2 [7680/30000 (26%)]	Loss: 0.000000
Train Epoch: 2 [10240/30000 (34%)]	Loss: 0.000000
Train Epoch: 2 [12800/30000 (43%)]	Loss: 0.000000
Train Epoch: 2 [15360/30000 (51%)]	Loss: 0.000000
Tr

In [17]:

savepath="save/pytorchNN=5"+order+str(order1)
torch.save(model, savepath);

In [18]:
#model = torch.load(PATH)
#model.eval()